In [94]:
# -*- coding: utf-8 -*-
"""
"""

import pickle
import numpy as np
import os
import datetime
import tensorflow as tf
from pathlib import Path

import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.optimizers import SGD
from keras.models import load_model

class DataManager:
    # Path to the WESAD dataset
    ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/WESAD'
    #ROOT_PATH = r'C:\WESAD'
    
    # pickle file extension for importing
    FILE_EXT = '.pkl'

    # Directory in project structure where model files are stored
    MODELS_DIR = os.path.join(Path().absolute(), 'src', 'models')

    # IDs of the subjects
    SUBJECTS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    
    # Label values defined in the WESAD readme
    BASELINE = 1
    STRESS = 2
    
    FEATURE_KEYS =     ['max',  'min', 'mean', 'range', 'std']
    FEATURE_ACC_KEYS = ['maxx', 'maxy', 'maxz', 'mean', 'std']

    # Keys for measurements collected by the RespiBAN on the chest
    # minus the ones we don't want
    # RAW_SENSOR_VALUES = ['ACC','ECG','EDA','EMG','Resp','Temp']
    RAW_SENSOR_VALUES_C = ['ECG']     ##Added ECG and EMG 
    RAW_SENSOR_VALUES_W = ['ACC','BVP', 'TEMP','EDA']
    FEATURES_C = {
                'ecg_max': [], 'ecg_min': [], 'ecg_mean' : [] , 'ecg_range' : [], 'ecg_std' : [], \
                }

    FEATURES_W = {'a_mean': [], 'a_std': [], 'a_maxx': [], 'a_maxy': [], 'a_maxz': [], \
                  'b_mean': [], 'b_std': [], 'b_maxx': [], 'b_maxy': [], 'b_maxz': [], \
                  't_max': [],  't_min': [], 't_mean': [], 't_range': [], 't_std': [], \
                  'e_max': [],  'e_min': [], 'e_mean': [], 'e_range': [], 'e_std': [], \
                } 

    FEATURES = {
                  'ecg_max': [], 'ecg_min': [], 'ecg_mean' : [] , 'ecg_range' : [], 'ecg_std' : [], \
                  'a_mean': [], 'a_std': [], 'a_maxx': [], 'a_maxy': [], 'a_maxz': [], \
                  'b_mean': [], 'b_std': [], 'b_max': [], 'b_range': [], 'b_min': [], \
                  't_max': [],  't_min': [], 't_mean': [], 't_range': [], 't_std': [], \
                  'e_max': [],  'e_min': [], 'e_mean': [], 'e_range': [], 'e_std': [], \
        
                }

                # Check 
    STRESS_FEATURES = {
                  'ecg_max': [], 'ecg_min': [], 'ecg_mean' : [] , 'ecg_range' : [], 'ecg_std' : [], \
                  'a_mean': [], 'a_std': [], 'a_maxx': [], 'a_maxy': [], 'a_maxz': [], \
                  'b_mean': [], 'b_std': [], 'b_max': [], 'b_range': [], 'b_min': [], \
                  't_max': [],  't_min': [], 't_mean': [], 't_range': [], 't_std': [], \
                  'e_max': [],  'e_min': [], 'e_mean': [], 'e_range': [], 'e_std': [], \
        
                }
    
    # Dictionaries to store the two sets of data
    BASELINE_DATA = []
    STRESS_DATA = []
    
    # the file name for the last created model
    last_saved=''
    
    def __init__(self, ignore_empatica=True, ignore_additional_signals=True):         #dont know what empatica means
        # denotes that we will be excluding the empatica data 
        # after loading those measurements
        self.ignore_empatica = ignore_empatica

    def get_subject_path(self, subject):
        """ 
        Parameters:
        subject (int): id of the subject
        
        Returns:
        str: path to the pickle file for the given subject number
             iff the path exists 
        """
        
        # subjects path looks like data_set + '<subject>/<subject>.pkl'

        path = DataManager.ROOT_PATH + '/S' + str(subject) + '/S' + str(subject) + DataManager.FILE_EXT 

        #path = os.path.join(DataManager.ROOT_PATH, 'S'+ str(subject), 'S' + str(subject) + DataManager.FILE_EXT)
        
        print('Loading data for S'+ str(subject))
        return path
        #print('Path=' + path)
        #if os.path.isfile(path):
         #   return path
        #else:
         #   print(path)
          #  raise Exception('Invalid subject: ' + str(subject))

    def load(self, subject):
        """ 
        Loads and saves the data from the pkl file for the provided subject
        
        Parameters:
        subject (int): id of the subject
        
        Returns: Baseline and stress data
        dict: {{'EDA': [###, ..], ..}, 
               {'EDA': [###, ..], ..} }
        """
       
        # change the encoding because the data appears to have been
        # pickled with py2 and we are in py3
        with open(self.get_subject_path(subject), 'rb') as file:
            data = pickle.load(file, encoding='latin1')
            return self.extract_and_reform(data, subject)
    
    def load_all(self, subjects=SUBJECTS):
        """ 
        Parameters:
        subjects (list): subject ids to be loaded
        
        Returns:
        """
        for subject in subjects:
            self.load(subject)
                
    
    def extract_and_reform(self, data, subject):
        """ 
        Extracts and shapes the data from the pkl file
        for the provided subject.
        
        Parameters:
        data (dict): as loaded from the pickle file
        
        Returns: Baseline and stress data
        dict: {{'EDA': [###, ..], ..}, 
               {'EDA': [###, ..], ..} }
        """
                
        #if self.ignore_empatica:
         #   del data['signal']['wrist']
        
        baseline_indices = np.nonzero(data['label']==DataManager.BASELINE)[0]   
        stress_indices = np.nonzero(data['label']==DataManager.STRESS)[0]
        base = dict()
        stress = dict()
        
        for value in DataManager.RAW_SENSOR_VALUES_C: 
            base[value] = data['signal']['chest'][value][baseline_indices]
            stress[value] = data['signal']['chest'][value][stress_indices]

        for value in DataManager.RAW_SENSOR_VALUES_W:
          #a=np.ones(len(s2_data['signal']['wrist']['ACC']))*round(700/32)
            mult = 1
            y = np.ones(len(data['signal']['wrist'][value]))

            if(value == 'ACC'):
              mult = 700/32
            if (value == 'TEMP'):
              mult = 700/4
            if (value == 'BVP'):
              mult = 700/64
            if (value == 'EDA') :
              mult = 700/4

            y = y*round(mult)
            y = y.astype(int)
            y = y.tolist()

            x = np.repeat(data['signal']['wrist'][value],y,axis=0)
            
            x1 = x[baseline_indices]
            x2 = x[stress_indices]
            
            #base[value] = data['signal']['wrist'][value][baseline_indices]
            #stress[value] = data['signal']['wrist'][value][stress_indices] 

            base[value] = x1
            stress[value] = x2 

        DataManager.BASELINE_DATA.append(base)
        DataManager.STRESS_DATA.append(stress)
        
        return base, stress
    
    def get_stats(self, values, window_size=42000, window_shift=175):
        """ 
        Calculates basic statistics including max, min, mean, and std
        for the given data
        
        Parameters:
        values (numpy.ndarray): list of numeric sensor values
        
        Returns: 
        dict: 
        """
#        print("There are ",  values.size, " samples being considered.")
        num_features = values.size - window_size
#        print("Computing ", num_features , " feature values with window size" \
#              "of ", str(window_size) + "." )        
        max_tmp = []
        min_tmp = []
        mean_tmp = []
        dynamic_range_tmp = []
        std_tmp = []
        for i in range(0, num_features, window_shift):
            window = values[i:window_size + i]
            max_tmp.append(np.amax(window))
            min_tmp.append(np.amin(window))
            mean_tmp.append(np.mean(window))
            dynamic_range_tmp.append(max_tmp[-1] - min_tmp[-1])
            std_tmp.append(np.std(window))

        features = {}
        features['max'] = max_tmp
        features['min'] = min_tmp
        features['mean'] = mean_tmp
        features['range'] = dynamic_range_tmp
        features['std'] = std_tmp
        return features

    def get_features_for_acc(self, values, window_size=42000, window_shift=175):
        """ 
        Calculates statistics including mean and std
        for the given data and the peak frequency per axis and the
        body acceleration component (RMS)
        
        Parameters:
        values (numpy.ndarray): list of numeric sensor values [x, y, z]
        window_size (int): specifies size of the sliding window
        window_shift (int): Specifies the sliding window shift
        
        Returns: 
        dict: features for mean, max, std for given data
        """
        #print("There are ", len(values[:,1]), " samples being considered.")
        num_features = len(values[:,1]) - window_size
        #print("Computing ", num_features , " feature values with window size" \
        #              "of ", str(window_size) + "." )
        maxx_tmp = []
        maxy_tmp = []
        maxz_tmp = []
        mean_tmp = []
        std_tmp = []        
        for i in range(0, num_features, window_shift):
            windowx = values[i:window_size + i, 0]
            windowy = values[i:window_size + i, 1]
            windowz = values[i:window_size + i, 2]
                        
            meanx = np.mean(windowx)
            meany = np.mean(windowy)
            meanz = np.mean(windowz)
            mean_tmp.append( (meanx + meany + meanz) )

            stdx = np.std(windowx)
            stdy = np.std(windowy)
            stdz = np.std(windowz)
            std_tmp.append( (stdx + stdy + stdz) )
            
            maxx_tmp.append(np.amax(windowx))
            maxy_tmp.append(np.amax(windowy))
            maxz_tmp.append(np.amax(windowz))

        features = {}
        features['mean'] = mean_tmp
        features['std'] =  std_tmp
        features['maxx'] = maxx_tmp
        features['maxy'] = maxy_tmp
        features['maxz'] = maxz_tmp
        
        return features
    
    def compute_features(self, subjects=SUBJECTS, data=BASELINE_DATA, window_size=42000, window_shift=175):
        """ 
        Calculates features for the provided subjects given the data and
        window size/shift of interest. 
        
        Parameters:
        subjects (list): subject ids to be loaded
        data (list): List of dictionaries containing subjects as indices
                     and dictionaries with features for each 'ACC', 'EDA', 'Temp'
        window_size (int): specifies size of the sliding window
        window_shift (int): Specifies the sliding window shift
        Returns:
            
        """
        keys = list(DataManager.FEATURES.keys())
        print('Computing features..')
        for subject in subjects:
            print("\tsubject:", subject)
            index = subject - 2
            key_index = 0
            
            acc = self.get_features_for_acc(data[index]['ACC'], window_size, window_shift)
            for feature in DataManager.FEATURE_ACC_KEYS:
                #print('computed ', len(acc[feature]), 'windows for acc ', feature)
                DataManager.FEATURES[keys[key_index]].extend(acc[feature])
                key_index = key_index + 1
            
            eda = self.get_stats(data[index]['EDA'], window_size, window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print('computed ', len(eda[feature]), 'windows for eda ', feature)
                DataManager.FEATURES[keys[key_index]].extend(eda[feature])
                key_index = key_index + 1

            temp = self.get_stats(data[index]['TEMP'], window_size, window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print('computed ', len(temp[feature]), 'windows for temp ', feature)
                DataManager.FEATURES[keys[key_index]].extend(temp[feature])
                key_index = key_index + 1

           

            bvp = self.get_stats(data[index]['BVP'],window_size,window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print('computed',len(bvp[feature]),'windows for bvp'feature)
                DataManager.FEATURES[keys[key_index]].extend(bvp[feature])
                key_index = key_index + 1

            ecg = self.get_stats(data[index]['ECG'],window_size,window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print('computed',len(ecg[feature]), 'windows for ecg', feature)
                DataManager.FEATURES[keys[key_index]].extend(ecg[feature])
                key_index = key_index + 1

            
        return DataManager.FEATURES

    def compute_features_stress(self, subjects=SUBJECTS, data=STRESS_DATA, window_size=42000, window_shift=175):
        keys = list(DataManager.STRESS_FEATURES.keys())
        print('computing features..')    
        for subject in subjects:
            print("\tsubject:", subject)
            index = subject - 2
            key_index = 0
            
            acc = self.get_features_for_acc(data[index]['ACC'], window_size, window_shift)
            for feature in DataManager.FEATURE_ACC_KEYS:
                #print('computed ', len(acc[feature]), 'windows for acc ', feature)
                DataManager.STRESS_FEATURES[keys[key_index]].extend(acc[feature])
                key_index = key_index + 1
            
            eda = self.get_stats(data[index]['EDA'], window_size, window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print('computed ', len(eda[feature]), 'windows for eda ', feature)
                DataManager.STRESS_FEATURES[keys[key_index]].extend(eda[feature])
                key_index = key_index + 1

            temp = self.get_stats(data[index]['TEMP'], window_size, window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print('computed ', len(temp[feature]), 'windows for temp ', feature)
                DataManager.STRESS_FEATURES[keys[key_index]].extend(temp[feature])
                key_index = key_index + 1

            

            bvp = self.get_stats(data[index]['BVP'],window_size,window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print statement
                DataManager.STRESS_FEATURES[keys[key_index]].extend(bvp[feature])
                key_index = key_index + 1

            ecg = self.get_stats(data[index]['ECG'],window_size, window_shift)
            for feature in DataManager.FEATURE_KEYS:
                #print statement
                DataManager.STRESS_FEATURES[keys[key_index]].extend(ecg[feature])
                key_index = key_index + 1

        return DataManager.STRESS_FEATURES

    def get_train_and_test_data(self):
        X1 = []
        X2 = []
        for i in range(0, len(DataManager.FEATURES['a_mean'])):
            X1.append([DataManager.FEATURES['a_mean'][i], DataManager.FEATURES['a_std'][i],\
                       DataManager.FEATURES['a_maxx'][i], DataManager.FEATURES['a_maxy'][i],\
                       DataManager.FEATURES['a_maxz'][i], DataManager.FEATURES['e_max'][i],\
                       DataManager.FEATURES['e_min'][i],  DataManager.FEATURES['e_mean'][i],\
                       DataManager.FEATURES['e_range'][i],DataManager.FEATURES['e_std'][i],\
                       DataManager.FEATURES['t_max'][i],  DataManager.FEATURES['t_min'][i],\
                       DataManager.FEATURES['t_mean'][i], DataManager.FEATURES['t_range'][i],\
                       DataManager.FEATURES['t_std'][i], DataManager.FEATURES['b_max'][i],  DataManager.FEATURES['b_min'][i],\
                       DataManager.FEATURES['b_mean'][i], DataManager.FEATURES['b_range'][i],\
                       DataManager.FEATURES['b_std'][i], DataManager.FEATURES['ecg_max'][i],  DataManager.FEATURES['ecg_min'][i],\
                       DataManager.FEATURES['ecg_mean'][i], DataManager.FEATURES['ecg_range'][i],\
                       DataManager.FEATURES['ecg_std'][i]])
        #print(np.shape(X1))
        
        for i in range(0,  len(DataManager.STRESS_FEATURES['a_mean'])):
            X2.append([DataManager.FEATURES['a_mean'][i], DataManager.FEATURES['a_std'][i],\
                       DataManager.FEATURES['a_maxx'][i], DataManager.FEATURES['a_maxy'][i],\
                       DataManager.FEATURES['a_maxz'][i], DataManager.FEATURES['e_max'][i],\
                       DataManager.FEATURES['e_min'][i],  DataManager.FEATURES['e_mean'][i],\
                       DataManager.FEATURES['e_range'][i],DataManager.FEATURES['e_std'][i],\
                       DataManager.FEATURES['t_max'][i],  DataManager.FEATURES['t_min'][i],\
                       DataManager.FEATURES['t_mean'][i], DataManager.FEATURES['t_range'][i],\
                       DataManager.FEATURES['t_std'][i], DataManager.FEATURES['b_max'][i],  DataManager.FEATURES['b_min'][i],\
                       DataManager.FEATURES['b_mean'][i], DataManager.FEATURES['b_range'][i],\
                       DataManager.FEATURES['b_std'][i], DataManager.FEATURES['ecg_max'][i],  DataManager.FEATURES['ecg_min'][i],\
                       DataManager.FEATURES['ecg_mean'][i], DataManager.FEATURES['ecg_range'][i],\
                       DataManager.FEATURES['ecg_std'][i]])                
        #print(np.shape(X2))
        
        # initialize zero for base and 1 for stress
        y1 = [0] * len(X1)
        y2 = [1] * len(X2)
        # Now we need to concat the data between baseline and stress so that 
        # we can split it into training and test sets    
        X = np.concatenate((X1, X2), axis=0)
        #print(np.shape(X))
        
        y = np.concatenate((y1,y2), axis=0)
        #print(np.shape(y))
        #X_train, X_test, y_train, y_test = \
            #train_test_split(X, y, test_size=0.20, random_state=42)
        return (X, y)

    def normalize(self, data):
        scaler = MinMaxScaler(feature_range=(0, 1))
        return scaler.fit_transform(data)

    def scale_data(self, X_train, X_test, y_train, y_test):
        print("Scaling the data...")
        X, y = self.get_train_and_test_data()
        (X_train, X_test, y_train, y_test) = (X,X,y,y)
        X_train = self.normalize(X_train)
        X_test = self.normalize(X_test)
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
        return (X_train, X_test, y_train, y_test)

    def build_model(self):
        num_neurons = 20
        num_features = 20
        
        print('Building the LSTM NN...')

        model = Sequential()
        model.add(Dense(30,input_shape=(1, num_features),activation = 'relu'))
        #model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=True))
        model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=False))
        model.add(Dense(30, activation='sigmoid'))
        model.add(Dense(1, activation='sigmoid'))
        
        model = self.configure_learning(model)
        print(model.summary())
        return model

    def configure_learning(self, model):
        opt = SGD(lr=0.05)
        model.compile(loss='binary_crossentropy', optimizer=opt,\
                      metrics=['accuracy'])
        return model

    def train_model(self, model, X_train, X_test, y_train, y_test,\
                    batch_size=32, epochs=5):
        print('Training network...')
        model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(X_test, y_test))
        #print("inputs: " , model.input_shape)
        #print("outputs: ", model.output_shape)
        #print("actual inputs: ", np.shape(X_train))
        #print("actual outputs: ", np.shape(y_train))
        score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
        return (model, score, acc)

    def load_model(self, file_name=last_saved):
        # Load the model of interest
        print("Loading model:", file_name)
        file = (os.path.join(DataManager.MODELS_DIR, file_name))
        model_from_disc = load_model(file)
        return model_from_disc

    def save_model(self, model):
        now = datetime.datetime.now()
        # Make sure the datetime str has no special characters and no spaces
        DataManager.last_saved = str("model-" + \
                                     str(now.replace(microsecond=0)) +\
                                     ".h5").replace(" ", "").replace(":", "_")
        model.save(os.path.join(DataManager.MODELS_DIR, DataManager.last_saved))
        print("Saved model to disc:",\
              DataManager.last_saved)
        
   

    def create_network(self, epochs=5, batch_size=32):
        '''
        Builds, trains, and saves a model with the provided number of epochs
        and batch size.
        '''
        (X_train, X_test, y_train, y_test) = self.get_train_and_test_data()
        (X_train, X_test, y_train, y_test) = \
            self.scale_data(X_train, X_test, y_train, y_test)
        model = self.build_model()
        (model, score, acc) = self.train_model(model, X_train, X_test, y_train, y_test,\
                                 batch_size, epochs)
        self.save_model(model)
        
        #self.get_model_results(model, X_train, X_test, y_train, y_test)
        return (model, X_train, X_test, y_train, y_test)



In [95]:

  
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.signal as scisig
import scipy.stats
#import cvxEDA

In [96]:
manager = DataManager()

In [97]:
manager.load_all()

Loading data for S2
Loading data for S3
Loading data for S4
Loading data for S5
Loading data for S6
Loading data for S7
Loading data for S8
Loading data for S9
Loading data for S10
Loading data for S11


In [98]:
manager.compute_features();
manager.compute_features_stress();

Computing features..
	subject: 2
	subject: 3
	subject: 4
	subject: 5
	subject: 6
	subject: 7
	subject: 8
	subject: 9
	subject: 10
	subject: 11
computing features..
	subject: 2
	subject: 3
	subject: 4
	subject: 5
	subject: 6
	subject: 7
	subject: 8
	subject: 9
	subject: 10
	subject: 11


In [99]:
X,y = manager.get_train_and_test_data()


In [100]:
np.shape(X)

(68241, 25)

In [101]:
#Removing ECG as it is not sampled from Chest
#Using same values for now
XX_train = X[:,:-5]
XX_test = X[:,:-5]

In [102]:
np.shape(XX_train)

(68241, 20)

In [103]:
y_train = y #Using same values for now

In [104]:
y_test = y

In [105]:
manager.scale_data(XX_train, XX_test, y_train, y_test)

Scaling the data...


(array([[[0.21796381, 0.17525588, 0.19851427, ..., 0.73404255,
          0.84202719, 0.91174985]],
 
        [[0.2172404 , 0.17525588, 0.1981536 , ..., 0.73404255,
          0.8417597 , 0.91177292]],
 
        [[0.21633604, 0.17525588, 0.19779596, ..., 0.73404255,
          0.84142022, 0.91178664]],
 
        ...,
 
        [[0.66419259, 0.83635078, 0.76843004, ..., 0.4893617 ,
          0.80212298, 0.19656905]],
 
        [[0.66419259, 0.83635078, 0.7683641 , ..., 0.4893617 ,
          0.80227458, 0.1913193 ]],
 
        [[0.66419259, 0.83635078, 0.76829783, ..., 0.4893617 ,
          0.80251936, 0.18688425]]]),
 array([[[0.21796381, 0.17525588, 0.19851427, ..., 0.73404255,
          0.84202719, 0.91174985]],
 
        [[0.2172404 , 0.17525588, 0.1981536 , ..., 0.73404255,
          0.8417597 , 0.91177292]],
 
        [[0.21633604, 0.17525588, 0.19779596, ..., 0.73404255,
          0.84142022, 0.91178664]],
 
        ...,
 
        [[0.66419259, 0.83635078, 0.76843004, ..., 0.4893617 

In [106]:
tf.config.run_functions_eagerly(True)
XX_train = np.reshape(XX_train, (XX_train.shape[0], 1, XX_train.shape[1]))
XX_test = np.reshape(XX_test, (XX_test.shape[0], 1, XX_test.shape[1]))

In [107]:
model = manager.build_model()

manager.train_model(model, XX_train, XX_test, y_train, y_test,\
                    batch_size=64, epochs=5)

Building the LSTM NN...
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 1, 30)             630       
                                                                 
 lstm_9 (LSTM)               (None, 20)                4080      
                                                                 
 dense_12 (Dense)            (None, 30)                630       
                                                                 
 dense_13 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,371
Trainable params: 5,371
Non-trainable params: 0
_________________________________________________________________
None
Training network...
Epoch 1/5
   1/1067 [..............................] - ETA: 53s - loss: 0.7990 - accuracy: 0.3750

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


1067/1067 [==============================] - 40s 37ms/step - loss: 0.6437 - accuracy: 0.6504 - val_loss: 0.6415 - val_accuracy: 0.6515
Epoch 2/5
1067/1067 [==============================] - 45s 42ms/step - loss: 0.6396 - accuracy: 0.6515 - val_loss: 0.6380 - val_accuracy: 0.6515
Epoch 3/5
1067/1067 [==============================] - 45s 42ms/step - loss: 0.6372 - accuracy: 0.6515 - val_loss: 0.6402 - val_accuracy: 0.6515
Epoch 4/5
1067/1067 [==============================] - 45s 42ms/step - loss: 0.6369 - accuracy: 0.6517 - val_loss: 0.6358 - val_accuracy: 0.6515
Epoch 5/5
1067/1067 [==============================] - 16s 15ms/step - loss: 0.6306 - accuracy: 0.6515


(<keras.engine.sequential.Sequential at 0x7f545dbe2710>,
 0.6306315064430237,
 0.6515291333198547)